In [1]:
from hmmlearn import hmm
import numpy as np
import math

import warnings
warnings.filterwarnings('ignore')

#### Model

* We have the data about observations states are unknown.  
* * Observation 1 : walk -> shop -> clean
* * Observation 2 : clean -> clean -> clean
* There is a prior probability of a first state in sequence.
* Transition probability : Probability of a next state given current state (never symmetric in general)  
* Emission probability : Probability of observation given state  

These three are the model parameters, which we need to infer.
Our example is discrete HMM and emission probability has multinomial distribution.  
Note that it all sums up to 1.  

In case of continuous HMM emission probabilities can be a normal distribution.  
One example of observed entitity is electric power consumed.  https://nipunbatra.github.io/blog/2013/hmm_continuous.html

In [2]:
states = ('Rainny', 'Sunny')
observations = ('walk', 'shop', 'clean')

start_probability = {
    'Rainy':0.6,
    'Sunny':0.4
}


trainsition_probabilities = {
    'Rainy' : {'Rainy':0.7, 'Sunny':0.3},
    'Sunny' : {'Rainy':0.4, 'Sunny':0.6}
}

emissio_probabilites = {
    'Rainy':{
        'walk':0.1,
        'shop':0.4,
        'clean':0.5
    },
    'Sunny':{
        'walk':0.6,
        'shop':0.3,
        'clean':0.1
    }
}

In [3]:
start_prob = np.array([0.6, 0.4])
trans_prob = np.array([[0.7,0.3],
                                [0.4,0.6]])
emission_prob = np.array([[0.1, 0.4, 0.5],
                                [0.6, 0.3, 0.1]])

In [4]:
import numpy as np
from hmmlearn import hmm

model = hmm.MultinomialHMM(n_components=2)
model.startprob_ = start_prob
model.transmat_ = trans_prob
model.emissionprob_ = emission_prob

#### Foward algorithm - Handcoded

In [27]:
def forward_rec(observed_seq):
    memo = {}
    
    def sub_sol(state, seq_no):
        if seq_no == len(observed_seq):
            return 1
        tt = (state, seq_no)
        if tt in memo:
            prob = memo[tt]
        else:
            if seq_no == -1:
                prob = 0
                for i,p in enumerate(start_prob):
                    prob += p*sub_sol(i, seq_no + 1)
            else:
                prob = 1
                prob *= emission_prob[state][observed_seq[seq_no]]
                temp = 0
                for next_state in range(len(trans_prob)):
                    temp += trans_prob[state][next_state] * sub_sol(next_state, seq_no + 1)
                prob *= temp
            memo[tt] = prob

        return prob
    
    
    ans = sub_sol(0, -1)
    return ans

In [28]:
forward_rec([2,2,2])

0.045903999999999993

In [29]:
def forward_up(observed_seq):
    no_states = len(start_prob)

    dp = [[0 for _ in range(len(observed_seq))] for _ in range(no_states)]
    
    for i in range(len(observed_seq)-1, -1, -1):
        observation = observed_seq[i]    
        if i == len(observed_seq) - 1:
            for j in range(no_states):
                dp[j][i] = emission_prob[j][observation]
        else:
            for state in range(no_states):
                prob = 1
                prob *= emission_prob[state][observed_seq[i]]
                temp = 0
                for next_state in range(len(trans_prob)):
                    temp += trans_prob[state][next_state] * dp[next_state][i + 1]
                prob *= temp
                dp[state][i]=prob
    
    ans = 0
    for i in range(no_states):
        ans += start_prob[i] * dp[i][0]
    return ans  

In [30]:
forward_up([2,2,2])

0.045903999999999993

In [31]:
forward_up([0,1,2])

0.033612000000000003

In [32]:
forward_rec([1,1])

0.12959999999999999

#### Forward Algorithm

Given model we want to calculate probability of observing sequence.  Brute force approach will start from each state, calculate probability and sums them up. If there are N hidden states and T observations there can be total N^T possible sequence. Forward algorithm uses dynamic programming to reduce complexity. 

In [9]:
prob = math.exp(model.score([2,2,2]))
print "Probability of observing clean -> clean -> clean is", prob

Probability of observing clean -> clean -> clean is 0.045904


In [10]:
prob = math.exp(model.score([0,1,2]))
print "Probability of observing walk -> shop -> clean is", prob

Probability of observing walk -> shop -> clean is 0.033612


In [11]:
prob = math.exp(model.score([1,1]))
print "Probability of observing shop -> shop is", prob

Probability of observing shop -> shop is 0.1296


#### Backward algorithm - viterbi - Handcoded

In [54]:
def decode(observed_seq):
    """ Return path and probabilities"""
    
    no_states = len(start_prob)
    parent = [[None for _ in range(len(observed_seq))] for _ in range(no_states)]
    dp =  [[None for _ in range(len(observed_seq))] for _ in range(no_states)]
    
    prob = 0
    
    first_observation = observed_seq[0]
    for state in range(no_states):
        dp[state][0] = start_prob[state]*emission_prob[state][first_observation]
        parent[state][0] = -1
        
    for seq_no in range(1, len(observed_seq)):
        observation = observed_seq[seq_no]
        for state in range(no_states):
            prob = 0
            parent_x = -2
            for prev_state in range(no_states):
                temp_prob = dp[prev_state][seq_no-1] * trans_prob[prev_state][state] * emission_prob[state][observation]
                if temp_prob > prob:
                    prob = temp_prob
                    parent_x = prev_state
            dp[state][seq_no] = prob
            parent[state][seq_no] = parent_x
            
    final_state = -1
    final_prob = 0
    
    for i in range(len(dp)):
        temp_prob = dp[i][-1]
        if temp_prob > final_prob:
            final_state = i
            final_prob = temp_prob
            
    back_track = [final_state]
    seq = len(observed_seq) - 1
    pp = parent[final_state][seq]
    
    while pp!=-1:
        back_track.append(pp)
        seq -= 1
        pp = parent[pp][seq]
        
    return list(reversed(back_track)), final_prob
    
    

In [55]:
decode([1,2,0])

([0, 0, 1], 0.015119999999999998)

In [57]:
decode([2,2,2])

([0, 0, 0], 0.036749999999999998)

In [56]:
decode([2,2,2,2,0,0,1,1,2])

([0, 0, 0, 0, 1, 1, 0, 0, 0], 1.3069123199999998e-05)

#### Decoding 

Given observed sequence, we want to find out most probable state sequence.  One way would be as in forward algorithm compute probability of each possible sequence and return the one with highest probability.  
Efficient algorithm here is Viterbi's algorithm which again is based on dynamic programming.  

In [47]:
# Predict the optimal sequence of internal hidden state
X = [1,2,0]
ans = model.decode(X)
print ans
print np.exp(ans[0])

(-4.19173690823075, array([0, 0, 1]))
0.01512


We had observed shop -> clean -> walk.  
States suggested are Rainy, Rainy, Sunny

In [58]:
X = [2,2,2]
ans = model.decode(X)
print ans
print np.exp(ans[0])

(-3.3036170533232916, array([0, 0, 0]))
0.03675


We had observed clean -> clean -> clean.  
States suggested are Rainy -> Rainy -> Rainy.  

In [59]:
X = [2,2,2,2,0,0,1,1,2]
ans = model.decode(X)
print ans
print np.exp(ans[0])

(-11.24525811750575, array([0, 0, 0, 0, 1, 1, 0, 0, 0]))
1.30691232e-05


We had observed clean -> clean -> clean -> clean -> walk -> walk -> shop -> shop -> clean.  
States suggested are Rainy -> Rainy -> Rainy -> Rainy -> Sunny -> Suny -> Rainy -> Rainy -> Rainy

#### Training handcoded

TODO : It is a form of EM algorithm.   
1) You assume some parameter values   
2) Repeat following until convergence :  
    * Determine probable state paths
    * Transition probability can be calculated by no of such a trasaction made
    * Emission probability can be calcuated by no of emissions made given transaction
    * Thus we re-estimate parameter values
3) We repeat two until convergence.

#### Training 

Given dataset of observed sequence we want to learn model parameters.  Algorithm used here is called forward-backward algorithm.  

###### Generating sample data

In [15]:
data = []
for i in range(200):
    X, Z = model.sample(6)
    data.append(X)

We now have 20 possible sequences of length 6.  

In [16]:
for i in range(4):
    print data[i]

[0 0 0 0 1 2]
[1 1 1 0 0 1]
[2 0 1 0 2 1]
[1 2 1 0 1 0]


###### Fitting Model

In [17]:
mm = hmm.MultinomialHMM(n_components=2)
mm.fit(data)

MultinomialHMM(algorithm='viterbi',
        init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
        n_components=2, n_iter=10,
        params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
        random_state=<mtrand.RandomState object at 0x7ffbdc1026e0>,
        startprob=None, startprob_prior=1.0, thresh=0.01, transmat=None,
        transmat_prior=1.0)

In [18]:
mm.emissionprob_

array([[ 0.14152644,  0.61370742,  0.24476614],
       [ 0.51719395,  0.09216437,  0.39064168]])

In [19]:
mm.startprob_

array([ 0.51716287,  0.48283713])

In [20]:
mm.transmat_

array([[ 0.57500492,  0.42499508],
       [ 0.52700236,  0.47299764]])

References :  
* https://web.stanford.edu/~jurafsky/slp3/A.pdf  
* https://medium.com/@kangeugine/hidden-markov-model-7681c22f5b9  

Further reading :  
* We had used library called 'hmmlearn' which seems to be deprecated.  
Library named 'hmms' have great example notebook.  

* https://github.com/lopatovsky/HMMs/blob/master/hmms.ipynb
